In [1]:
import sys
!{sys.executable} -m pip install tracery
!{sys.executable} -m pip install pronouncing

In [2]:
import tracery
import pronouncing
import random
from tracery.modifiers import base_english

In [3]:
import json

with open ('adjs.json', 'r') as input_file:
    adjs = []
    for adj in json.load(input_file)['adjs']:
        if pronouncing.phones_for_word(adj):
            if pronouncing.syllable_count(pronouncing.phones_for_word(adj)[0]) < 3:
                adjs.append(adj.lower())
    
with open ('personal_nouns.json', 'r') as input_file:
    nouns = [noun.lower() for noun in json.load(input_file)['personalNouns']]
    
with open ('firstNames.json', 'r') as input_file:
    names = [noun.lower() for noun in json.load(input_file)['firstNames']]
    
with open ('character.json', 'r') as input_file:
    characters = []
    for character in json.load(input_file)['characters']:
        characters.append(character["name"].lower())
        characters += character["synonyms"]
        
with open ('occupations.json', 'r') as input_file:
    characters += [job.lower() for job in json.load(input_file)['occupations']]
        
with open ('anthropogenic_features.json', 'r') as input_file:
    locations = ['the ' + entry.lower() for entry in json.load(input_file)['entries']]
    
with open ('us_cities.json', 'r') as input_file:
    locations += [city['city'].lower() for city in json.load(input_file)['cities']]
    
with open ('atus_activities.json', 'r') as input_file:
    activities = []
    for activity in json.load(input_file)['categories']:
        if "examples" in activity:
            for example in activity['examples']:
                if 'hh' not in example:
                    if '(' in example:
                        activities.append(example[:example.index("(",)])
                    else:
                        activities.append(example)

In [4]:
def return_syll_rhyme(text):
    syllabic_rhymes = [rhyme for rhyme in pronouncing.rhymes(text) if pronouncing.syllable_count(pronouncing.phones_for_word(text)[0]) == pronouncing.syllable_count(pronouncing.phones_for_word(rhyme)[0])]
    if syllabic_rhymes == []:
        return ""
    else: 
        return random.choice(syllabic_rhymes)
    
def return_noun_rhyme(text):
    noun_rhymes = []
    for rhyme in pronouncing.rhymes(text.split().pop()):
        if rhyme in nouns:
            noun_rhymes.append(rhyme)
    if noun_rhymes == []:
        return random.choice(nouns)
    else: 
        return random.choice(noun_rhymes)
    
def return_adj_rhyme(text):
    adj_rhymes = []
    for rhyme in pronouncing.rhymes(text.split().pop()):
        if rhyme in adjs:
            adj_rhymes.append(rhyme)
    if adj_rhymes == []:
        return random.choice(adjs)
    else: 
        return random.choice(adj_rhymes)

def return_allit_noun(text):
    if pronouncing.phones_for_word(text) == []:
        return random.choice(nouns)
    allit = []
    for noun in nouns:
        if pronouncing.phones_for_word(noun):
            if pronouncing.phones_for_word(noun)[0].split()[0] == pronouncing.phones_for_word(text)[0].split()[0]:
                allit.append(noun)
    if allit == []:
        return random.choice(noun)
    else: 
        return random.choice(allit)
    
def return_allit_name(text):
    if pronouncing.phones_for_word(text) == []:
        return random.choice(names)
    allit = []
    for name in names:
        if pronouncing.phones_for_word(name):
            if pronouncing.phones_for_word(name)[0].split()[0] == pronouncing.phones_for_word(text)[0].split()[0]:
                allit.append(name)
    if allit == []:
        return random.choice(names)
    else: 
        return random.choice(allit)
    
def return_allit_adj(text):
    if pronouncing.phones_for_word(text) == []:
        return random.choice(adjs)
    allit = []
    for adj in adjs:
        if pronouncing.phones_for_word(adj):
            if pronouncing.phones_for_word(adj)[0].split()[0] == pronouncing.phones_for_word(text)[0].split()[0]:
                allit.append(adj)
    if allit == []:
        return random.choice(adjs)
    else: 
        return random.choice(allit)

def return_rand_word(text):
    text = text.split()
    return random.choice(text)

def return_first_word(text):
    text = text.split()
    return text[0]
    
modifiers = {
    "syll_rhyme": return_syll_rhyme,
    "n_rhyme": return_noun_rhyme,
    "adj_rhyme": return_adj_rhyme,
    "n_allit": return_allit_noun,
    "name_allit": return_allit_name,
    "adj_allit": return_allit_adj,
    "rand_word": return_rand_word,
    "first_word": return_first_word
}

In [8]:
for i in range(7):
    rules = {
        "character": characters,
        "set_name_root": adjs,
        "location": random.choice(locations),
        "character_description": ["#name_root.syll_rhyme#-#character_description#", ""],
        "character_name": "#[name_root:#set_name_root#]character_description##name_root# #[first_name:#name_root.name_allit#]first_name#",
        "intro": ["#character_name# #employment#"],
        "employment": "\n(the #[occupation:#character#]occupation##addit_employment#",
        "addit_employment": ["\n    and #[occupation:#character#]occupation##addit_employment#", " of #location##character_info#"],
        "character_info": "\nwith #[object_name:#occupation.n_rhyme#]object_name.a# #object_desc##addit_character_info#",
        "addit_character_info": ["\n    and #[object_name:#occupation.n_rhyme#]object_name.a# #object_desc##addit_character_info#", ")"],
        "object_desc": ["so #object_name.adj_rhyme#", "of #object_name.n_rhyme#", "so #object_name.adj_allit#", "of #object_name.n_allit#", ],
        "action": random.choice(activities),
        "origin": ["#intro# \nwent #action#\nand sang a little tune"]
    }
    grammar = tracery.Grammar(rules)
    grammar.add_modifiers(base_english)
    grammar.add_modifiers(modifiers)

    print(grammar.flatten("#origin#") + '\n')

219
faulting-vaulting vincent 
(the coating worker of johnson city
with a pie so primal) 
went leading story time at library 
and sang a little tune

378
sprinted-hinted-tinted-sprinted-printed patrick 
(the extraction worker
    and stationary engineer
    and helper of albany
with a trave of kinsman) 
went talking on phone to a principal
and sang a little tune

151
-piquant paige 
(the clinical laboratory technologist
    and shaper of tampa
with a pell-mell of tentorium) 
went planting flowers 
and sang a little tune

174
running reagan 
(the self of davenport
with an elf so excess
    and an elf of exquisite) 
went unloading groceries/shopping items
and sang a little tune

242
midsized-outsized ashlyn 
(the exhibit designer
    and school social worker of pearl city
with a percuss so pastel) 
went texting with a friend
and sang a little tune

161
vagrant-vagrant-fragrant francisco 
(the woodworker of denver
with a headstall of teaser) 
went watching TV/DVDs on computer 
and sang a 